In [1]:
import os
from tqdm import tqdm
import numpy as np
import tiktoken
from datasets import load_dataset # huggingface datasets

In [2]:
dataset = load_dataset("json", data_files='./data/json/dev.json')

Found cached dataset json (/Users/bm/.cache/huggingface/datasets/json/default-14dc56ae28f0b495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
# owt by default only contains the 'train' split, so create a test split
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=2357, shuffle=True)
split_dataset['val'] = split_dataset.pop('test') # rename the test split to val

Loading cached split indices for dataset at /Users/bm/.cache/huggingface/datasets/json/default-14dc56ae28f0b495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-e65036e96329467f.arrow and /Users/bm/.cache/huggingface/datasets/json/default-14dc56ae28f0b495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-507da4f4a54b4983.arrow


In [18]:
from sentence_transformers import SentenceTransformer
# model_name = 'all-MiniLM-L6-v2'
# model = SentenceTransformer(model_name)
# we now want to tokenize the dataset. first define the encoding function (gpt2 bpe)
def process(example):
    global model
    if 'model' not in globals():
        model_name = 'all-MiniLM-L6-v2'
        model = SentenceTransformer(model_name)

    embds = model.encode(example['sentences'])
    print("after encode")
    out = {'id': example['id'], 'labels': example['labels'], 'embeddings': embds}
    return out

sample_dataset = split_dataset['train'].select(range(10))
# tokenize the dataset
tokenized = sample_dataset.map(
    process,
    # remove_columns=['sentences'],
    desc="tokenizing the splits",
    num_proc=7,
)


tokenizing the splits (num_proc=7):   0%|          | 0/10 [00:00<?, ? examples/s]

Process ForkPoolWorker-100:
Process ForkPoolWorker-98:
Process ForkPoolWorker-102:
Process ForkPoolWorker-101:
Process ForkPoolWorker-96:
Process ForkPoolWorker-97:
Process ForkPoolWorker-99:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/bm/miniforge3/envs/cats/lib/python3.10/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/bm/miniforge3/envs/cats/lib/python3.10/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/bm/miniforge3/envs/cats/lib/python3.10/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/bm/miniforge3/envs/cats/lib/python3.10/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/bm/miniforge3/en

TimeoutError: 

In [4]:
from datasets import load_dataset
from multiprocessing import Pool, cpu_count
from tokenizer import process

dataset = load_dataset("json", data_files='./data/json/dev.json')



Found cached dataset json (/Users/bm/.cache/huggingface/datasets/json/default-14dc56ae28f0b495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

72354

In [6]:
dataset['train']['sentences'][1:3]

[['According to the "Epic of Sundiata", Kouroukan Fouga or Kurukan Fuga was the constitution of the Mali Empire created after the Battle of Krina (1235) by an assembly of nobles to create a government for the newly established empire.',
  'According to oral tradition of the "griot" poets of Mali and Guinea, the Kouroukan Fouga established the federation of Mandinka clans under one government, outlined how it would operate and established the laws by which the people would live.',
  'The name "Kurukan Fuga" is a toponym, translating to "clearing on granite / lateritic rock", referring to the plain near the town of Ka-ba (present day Kangaba) where the narrative has Sundiata Keita present the charter.',
  'The "Manden Charter, proclaimed in Kurukan Fuga", was inscribed in 2009 (4.COM) on the Representative List of the Intangible Cultural Heritage of Humanity kept by UNESCO.',
  'The earliest extant versions of the "Epic of Sundiata" were collected in the 1890s, and the first close transc

In [ ]:

# Tokenize the dataset using multiprocessing
def tokenize_dataset(dataset):
    num_processes = cpu_count() - 1
    pool = Pool(processes=num_processes)

    examples = dataset['train']['sentences']
    examples_per_process = len(examples) // num_processes

    results = []

    for i in range(num_processes):
        start_idx = i * examples_per_process
        end_idx = (i+1) * examples_per_process if i < num_processes - 1 else len(examples)

        batch = examples[start_idx:end_idx]
        results.append(pool.map(process, batch))

    pool.close()
    pool.join()

    # Flatten the list of lists and return as a single list of examples
    flattened_results = [item for sublist in results for item in sublist]
    return flattened_results

tokenized_dataset = tokenize_dataset(dataset)